In [1]:
import sympy as syms
import numpy as np
import math

from sympy import symbols
from sympy import Matrix
from sympy import latex
from sympy import solve

In [ ]:
# Element Properties
E, A, I, L, P, lbda = symbols('E A I L P \lambda')
E = 70000
A = 100
L = 1000
I = 833.3
P = -1

In [ ]:
# Case 1: Simply Supported 2 Node Frame Element
# Reduced Linear Stiffness Matrix
K_lin = ((E*I)/(L))*Matrix([[4,0,2],[0, A/I,0],[2,0,4]])
K_lin

In [ ]:
# Reduced Geometric Stiffness Matrix
K_geo = (P/10)*Matrix([[4*L/3,0,-L/3],[0,0,0],[-L/3,0,4*L/3]])
K_geo

In [ ]:
# Reduced total stiffness matrix at bifurcation point
K_tau = K_lin + lbda*K_geo
K_tau

In [ ]:
# At the bifurcation point, the the stiffness matrix will be singular
detEqn =K_tau.det()
detEqn

In [ ]:
detEqn = detEqn.subs([(E,70000), (A,100), (L,1000), (I,833.3), (P,-1)]) 
detEqn

In [ ]:
lbdas = solve(detEqn,lbda)
lbdas

In [ ]:
# Critical Load
P_crit = min(lbdas)*P
P_crit

In [ ]:
# Case 2: Cantilever 2 Node Frame Element (Applied Load not given)
# Element Properties
E, A, I, L, P, lbda = symbols('E A I L P \lambda')


In [ ]:
F = syms.symbols('F') # Unknown Applied Critical Load (includes lambda too)
# Reduced Linear Stiffness Matrix
K_lin = ((E*I)/(L))*Matrix([[A/I,0,0],[0,12*(L**2),-6*L],[0,-6*L,4]])
K_lin

In [ ]:
# Reduced Geometric stiffness matrix 
K_geo = (F/10)*Matrix([[0,0,0],[0,12/L,-1],[0,-1,4*L/3]])
K_geo

In [ ]:
# Total Stiffness Matrix at bifurcation point
K_tau = K_lin + K_geo # lambda is multiplied with F and assumed to be the possible critical load
K_tau

In [ ]:
# Stiffness will be singular at bifurcation point
detEqn = K_tau.det()
detEqn

In [ ]:
force = syms.solve(detEqn,F)
force

In [27]:
# Critical Load
def criticalLoad(arr):
    for i in range(0,len(arr)):
        if abs(arr[i]) < abs(arr[0]):
            return arr[i]
    return arr[0]

f_crit = criticalLoad(force)
f_crit

NameError: name 'force' is not defined

In [ ]:
# Dispalcement required to make the elements buckle
# Node 2 (Free End) DOFs
u,w,phi = syms.symbols('u w \phi')
# Nodal Force Matrix
f_mat = syms.Matrix([[f_crit],[0],[0]])
# Reduced system nodal displacement vector
u_mat = syms.Matrix([[u],[w],[phi]])
f_mat = K_tau*u_mat
f_mat

In [ ]:
# Solving for displacement
syms.solve(f_mat[0] - f_crit,u)

In [ ]:
# Displacement from Euler's equation
f_euler = (math.pi**2)*E*I/(4*(L**2)) # Effective length for the given boundary conditions is 2L
f_euler

In [2]:
# 2 Shear Rigid Beam Elements (2 nodes per element)
# One end (node 1) fixed/clamped
# Second end (node 2) can slide horizontally in a guide
# #DOFs per element: 4 (bending delfection and rotation are the DOFs in one node)
# Boundary Conditions:
# Node 1 is fixed so w1 and phi1 = 0
# Node 3 -> w3 = ph3 = 0

# System Parameters
E, I, a, F = symbols('E I a F') # Length of each element: a/2

In [15]:
# Linear Stiffness Matrix for the reduced system based on the given boundary conditions
K_lin = ((2*E*I)/(a))*Matrix([[96/(a**2),0],[0,8]])
K_lin

Matrix([
[192*E*I/a**3,        0],
[           0, 16*E*I/a]])

In [16]:
# Geometric Stiffness Matrix for the reduced system based on the given boundary conditions
K_geo = ((F)/(10))*Matrix([[48/(a),0],[0,4*a/3]])
K_geo
# F = lambda*P, P is the applied force

Matrix([
[24*F/(5*a),        0],
[         0, 2*F*a/15]])

In [17]:
# Reduced Stiffness Matrix at the bifurcation point
K_tau = K_geo + K_lin
K_tau

Matrix([
[192*E*I/a**3 + 24*F/(5*a),                   0],
[                        0, 16*E*I/a + 2*F*a/15]])

In [18]:
# At the bifurcation point, the stiffness matrix becomes singular
# Determinant of Stiffness Matrix
det = K_tau.det()
det

(76800*E**2*I**2 + 2560*E*F*I*a**2 + 16*F**2*a**4)/(25*a**4)

In [30]:
# Solving for F
f = syms.solve(det,F)
f[1]

-40*E*I/a**2

In [31]:
P_crit = f[1]
P_crit

-40*E*I/a**2

In [26]:
# Theoretical buckling load computed according to Euler Buckling Theory
P_euler = -(4*(math.pi)**2)*(E*I)/(a**2)
P_euler

-39.4784176043574*E*I/a**2

In [32]:
# Error in numerical approximation
error = abs(P_crit - P_euler)/P_euler
error = error*100
error

-1.32118364233778*a**2*Abs(E*I/a**2)/(E*I)